In [1]:
#### for import clean data

import cv2
import os
import numpy as np
import pandas as pd
path = "C:/Users/Michael/Desktop/Berkeley/FinTech/DR/fintech_DR/images"
cleandata = pd.read_csv("C:/Users/Michael/Desktop/Berkeley/FinTech/DR/clean data.csv")
filenames = np.empty(1500)
filenames = cleandata["filename"]





In [2]:
#### read images and flat it

images = np.empty(len(filenames), dtype = object ) 
feature = []

for i in range(len(filenames)):              
    images[i] = cv2.imread(path + "/" + filenames[i])
    images[i] = cv2.resize(images[i], (512,512))
    #feature[i] = images[i].flatten()
    feature.extend(images[i])
feature_mat = np.matrix(np.reshape(feature, (1500,512*512*3)))
#feature_df = pd.DataFrame(feature)
feature_df = pd.DataFrame(feature_mat)




In [3]:
#### combining label and feature

fullset = pd.merge(cleandata, feature_df, left_index=True, right_index=True)
del fullset['Unnamed: 0']
fullset.head()


,filename,sight_threaten,0,1,2,3,4,5,6,7,...,786422,786423,786424,786425,786426,786427,786428,786429,786430,786431
0,36_right.jpeg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,57_left.jpeg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,60_left.jpeg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,78_right.jpeg,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,81_right.jpeg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
#### training and testing set split

from sklearn.model_selection import train_test_split 
from keras.utils import to_categorical
import random


random.seed( 30 )

y_train, y_test = train_test_split(fullset.iloc[:,0:2], test_size=0.2)



####################################

q = pd.DataFrame(y_train)
a = q[q["sight_threaten"].isin([0])]
b = q[q["sight_threaten"].isin([1])]


random.seed( 30 )
boota = np.random.choice(a["filename"], replace = True, size = 600)
bootb = np.random.choice(b["filename"], replace = True, size = 600)


### later use boot_df
boota_df = pd.DataFrame(boota, columns = ["filename"])
bootb_df = pd.DataFrame(bootb, columns = ["filename"])


train_filenames_df = pd.concat([boota_df, bootb_df])
y_new_train = pd.merge(train_filenames_df, cleandata.iloc[:,1:3], how = "left", on = ["filename", "filename"])
x_new_train = pd.merge(train_filenames_df, fullset, how = "left", on = ["filename", "filename"])
x_test = pd.merge(y_test.iloc[:,0:1], fullset, how = "left", on = ["filename", "filename"])

del x_new_train['sight_threaten']
del x_new_train['filename']
del x_test['sight_threaten']
del x_test['filename']
del fullset

#################################### 



# Onehot encoding for catogory
y_new_trainOneHot = to_categorical(y_new_train.iloc[:,1])
y_testOneHot = to_categorical(y_test.iloc[:,1])




Using TensorFlow backend.


In [5]:
x_test.head()

,0,1,2,3,4,5,6,7,8,9,...,786422,786423,786424,786425,786426,786427,786428,786429,786430,786431
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,1,1,1,1,1,...,4,1,4,2,0,3,1,1,4,2
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
#### PCA to do dimension reduction

from sklearn.decomposition import PCA
import time

n = 100
t0 = time.time()
pca = PCA(n_components=n, whiten=True).fit(x_new_train)
print("pca done %0.3fs" % (time.time() - t0))

print(sum(pca.explained_variance_ratio_))

t0 = time.time()
x_train_pca = pca.transform(x_new_train)
x_test_pca  = pca.transform(x_test)
del x_test
del x_new_train
print("data set to pca done %0.3fs" % (time.time() - t0))

pca done 714.214s
0.9842700555769404
data set to pca done 73.671s


In [7]:
### CNN 
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D




model = Sequential()


model.add(Dense(512, input_dim = n, activation = "relu"))

model.add(Dropout(0.4))


model.add(Dense(512, activation = "relu"))

model.add(Dropout(0.4))


model.add(Dense(512, activation = "relu"))

model.add(Dropout(0.4))

model.add(Dense(512, activation = "relu"))

model.add(Dropout(0.4))

model.add(Dense(512, activation = "relu"))

model.add(Dropout(0.4))

model.add(Dense(512, activation = "relu"))

model.add(Dropout(0.4))

model.add(Dense(512, activation = "relu"))

model.add(Dropout(0.4))

model.add(Dense(512, activation = "relu"))

model.add(Dropout(0.4))

model.add(Dense(512, activation = "relu"))

model.add(Dropout(0.4))





model.add(Dense(2, activation = 'softmax'))

model.compile(loss = "categorical_crossentropy",
             optimizer = "adam",
             metrics = ["accuracy"])
         

# maybe should change to focal loss?


model.summary()






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               51712     
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_____________________

In [8]:
random.seed( 30 )
model.fit(x = x_train_pca, y = y_new_trainOneHot, batch_size = 100, epochs = 1000, validation_split = 0.1, verbose = 2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 1080 samples, validate on 120 samples
Epoch 1/1000
 - 4s - loss: 0.7189 - acc: 0.5083 - val_loss: 0.7356 - val_acc: 0.0000e+00
Epoch 2/1000
 - 0s - loss: 0.7015 - acc: 0.5315 - val_loss: 0.6936 - val_acc: 0.4250
Epoch 3/1000
 - 0s - loss: 0.7051 - acc: 0.5287 - val_loss: 0.7178 - val_acc: 0.0000e+00
Epoch 4/1000
 - 0s - loss: 0.6952 - acc: 0.5352 - val_loss: 0.7501 - val_acc: 0.0000e+00
Epoch 5/1000
 - 0s - loss: 0.6897 - acc: 0.5750 - val_loss: 0.7289 - val_acc: 0.0000e+00
Epoch 6/1000
 - 0s - loss: 0.6923 - acc: 0.5426 - val_loss: 0.7524 - val_acc: 0.0000e+00
Epoch 7/1000
 - 0s - loss: 0.6848 - acc: 0.5583 - val_loss: 0.7470 - val_acc: 0.0000e+00
Epoch 8/1000
 - 0s - loss: 0.6827 - acc: 0.5583 - val_loss: 0.6731 - val_acc: 0.9583
Epoch 9/1000
 - 0s - loss: 0.6696 - acc: 0.5741 - val_loss: 0.7590 - val_acc: 0.1333
Epoch 10/1000
 - 0s - loss: 0.6398 - acc: 0.6074 - val_loss: 0.7562 - 

Epoch 92/1000
 - 0s - loss: 0.0047 - acc: 0.9963 - val_loss: 0.3103 - val_acc: 0.9750
Epoch 93/1000
 - 0s - loss: 0.0141 - acc: 0.9972 - val_loss: 0.2237 - val_acc: 0.9750
Epoch 94/1000
 - 0s - loss: 0.0089 - acc: 0.9963 - val_loss: 0.2366 - val_acc: 0.9750
Epoch 95/1000
 - 0s - loss: 0.0071 - acc: 0.9981 - val_loss: 0.1968 - val_acc: 0.9750
Epoch 96/1000
 - 0s - loss: 0.0165 - acc: 0.9981 - val_loss: 0.2734 - val_acc: 0.9667
Epoch 97/1000
 - 0s - loss: 0.0157 - acc: 0.9954 - val_loss: 0.1421 - val_acc: 0.9750
Epoch 98/1000
 - 0s - loss: 0.0335 - acc: 0.9944 - val_loss: 0.3029 - val_acc: 0.9750
Epoch 99/1000
 - 0s - loss: 0.0214 - acc: 0.9963 - val_loss: 0.0969 - val_acc: 0.9750
Epoch 100/1000
 - 0s - loss: 0.0084 - acc: 0.9981 - val_loss: 0.1296 - val_acc: 0.9750
Epoch 101/1000
 - 0s - loss: 0.0155 - acc: 0.9972 - val_loss: 0.2238 - val_acc: 0.9750
Epoch 102/1000
 - 0s - loss: 0.0035 - acc: 0.9991 - val_loss: 0.2958 - val_acc: 0.9750
Epoch 103/1000
 - 0s - loss: 0.0085 - acc: 0.9972 -

Epoch 187/1000
 - 0s - loss: 0.0158 - acc: 0.9972 - val_loss: 0.3572 - val_acc: 0.9667
Epoch 188/1000
 - 0s - loss: 0.0129 - acc: 0.9972 - val_loss: 0.3358 - val_acc: 0.9750
Epoch 189/1000
 - 0s - loss: 0.0199 - acc: 0.9954 - val_loss: 0.1270 - val_acc: 0.9667
Epoch 190/1000
 - 0s - loss: 0.0088 - acc: 0.9981 - val_loss: 0.1557 - val_acc: 0.9667
Epoch 191/1000
 - 0s - loss: 0.0096 - acc: 0.9954 - val_loss: 0.3605 - val_acc: 0.9667
Epoch 192/1000
 - 0s - loss: 0.0211 - acc: 0.9954 - val_loss: 0.3083 - val_acc: 0.9667
Epoch 193/1000
 - 0s - loss: 0.0035 - acc: 0.9991 - val_loss: 0.2129 - val_acc: 0.9667
Epoch 194/1000
 - 0s - loss: 0.0081 - acc: 0.9981 - val_loss: 0.3208 - val_acc: 0.9667
Epoch 195/1000
 - 0s - loss: 0.0026 - acc: 0.9991 - val_loss: 0.3541 - val_acc: 0.9667
Epoch 196/1000
 - 0s - loss: 0.0013 - acc: 1.0000 - val_loss: 0.3210 - val_acc: 0.9750
Epoch 197/1000
 - 0s - loss: 0.0020 - acc: 0.9991 - val_loss: 0.3261 - val_acc: 0.9750
Epoch 198/1000
 - 0s - loss: 0.0279 - acc: 

 - 0s - loss: 0.0031 - acc: 1.0000 - val_loss: 0.2972 - val_acc: 0.9750
Epoch 282/1000
 - 0s - loss: 0.0173 - acc: 0.9991 - val_loss: 0.4030 - val_acc: 0.9750
Epoch 283/1000
 - 0s - loss: 0.0018 - acc: 0.9991 - val_loss: 0.4030 - val_acc: 0.9750
Epoch 284/1000
 - 0s - loss: 0.0230 - acc: 0.9972 - val_loss: 0.3465 - val_acc: 0.9750
Epoch 285/1000
 - 0s - loss: 0.0405 - acc: 0.9935 - val_loss: 0.4165 - val_acc: 0.9667
Epoch 286/1000
 - 0s - loss: 0.0527 - acc: 0.9787 - val_loss: 0.2939 - val_acc: 0.9750
Epoch 287/1000
 - 0s - loss: 0.0480 - acc: 0.9898 - val_loss: 0.4030 - val_acc: 0.9750
Epoch 288/1000
 - 0s - loss: 0.0975 - acc: 0.9907 - val_loss: 0.4076 - val_acc: 0.9750
Epoch 289/1000
 - 0s - loss: 0.0259 - acc: 0.9944 - val_loss: 0.4150 - val_acc: 0.9667
Epoch 290/1000
 - 0s - loss: 0.0118 - acc: 0.9981 - val_loss: 0.5256 - val_acc: 0.9667
Epoch 291/1000
 - 0s - loss: 0.0312 - acc: 0.9944 - val_loss: 0.5016 - val_acc: 0.9667
Epoch 292/1000
 - 0s - loss: 0.0264 - acc: 0.9954 - val_lo

 - 0s - loss: 0.0153 - acc: 0.9991 - val_loss: 0.3879 - val_acc: 0.9750
Epoch 376/1000
 - 0s - loss: 6.3623e-04 - acc: 1.0000 - val_loss: 0.4038 - val_acc: 0.9750
Epoch 377/1000
 - 0s - loss: 0.0125 - acc: 0.9981 - val_loss: 0.4106 - val_acc: 0.9667
Epoch 378/1000
 - 0s - loss: 0.0010 - acc: 1.0000 - val_loss: 0.4316 - val_acc: 0.9667
Epoch 379/1000
 - 0s - loss: 0.0061 - acc: 0.9991 - val_loss: 0.4179 - val_acc: 0.9667
Epoch 380/1000
 - 0s - loss: 0.0039 - acc: 0.9991 - val_loss: 0.4100 - val_acc: 0.9667
Epoch 381/1000
 - 0s - loss: 0.0141 - acc: 0.9972 - val_loss: 0.3926 - val_acc: 0.9667
Epoch 382/1000
 - 0s - loss: 0.0026 - acc: 0.9991 - val_loss: 0.4072 - val_acc: 0.9667
Epoch 383/1000
 - 0s - loss: 0.0067 - acc: 0.9963 - val_loss: 0.4140 - val_acc: 0.9667
Epoch 384/1000
 - 0s - loss: 0.0130 - acc: 0.9963 - val_loss: 0.2734 - val_acc: 0.9750
Epoch 385/1000
 - 0s - loss: 0.0059 - acc: 0.9991 - val_loss: 0.3848 - val_acc: 0.9750
Epoch 386/1000
 - 0s - loss: 0.0032 - acc: 0.9981 - va

...

Epoch 563/1000
 - 0s - loss: 0.0191 - acc: 0.9981 - val_loss: 0.4524 - val_acc: 0.9667
Epoch 564/1000
 - 0s - loss: 0.0302 - acc: 0.9981 - val_loss: 0.5090 - val_acc: 0.9667
Epoch 565/1000
 - 0s - loss: 0.0149 - acc: 0.9991 - val_loss: 0.5373 - val_acc: 0.9667
Epoch 566/1000
 - 0s - loss: 0.0149 - acc: 0.9991 - val_loss: 0.5373 - val_acc: 0.9667
Epoch 567/1000
 - 0s - loss: 0.0152 - acc: 0.9991 - val_loss: 0.5373 - val_acc: 0.9667
Epoch 568/1000
 - 0s - loss: 0.0149 - acc: 0.9991 - val_loss: 0.5373 - val_acc: 0.9667
Epoch 569/1000
 - 0s - loss: 0.0149 - acc: 0.9991 - val_loss: 0.5373 - val_acc: 0.9667
Epoch 570/1000
 - 0s - loss: 0.0294 - acc: 0.9954 - val_loss: 0.4723 - val_acc: 0.9667
Epoch 571/1000
 - 0s - loss: 0.0189 - acc: 0.9944 - val_loss: 0.3510 - val_acc: 0.9667
Epoch 572/1000
 - 0s - loss: 0.0287 - acc: 0.9954 - val_loss: 0.3840 - val_acc: 0.9667
Epoch 573/1000
 - 0s - loss: 0.0095 - acc: 0.9963 - val_loss: 0.2776 - val_acc: 0.9750
Epoch 574/1000
 - 0s - loss: 0.0351 - acc: 

 - 0s - loss: 4.0669e-04 - acc: 1.0000 - val_loss: 0.2977 - val_acc: 0.9750
Epoch 658/1000
 - 0s - loss: 5.1789e-04 - acc: 1.0000 - val_loss: 0.2975 - val_acc: 0.9750
Epoch 659/1000
 - 0s - loss: 5.8588e-04 - acc: 1.0000 - val_loss: 0.3109 - val_acc: 0.9750
Epoch 660/1000
 - 0s - loss: 0.0037 - acc: 0.9981 - val_loss: 0.4030 - val_acc: 0.9750
Epoch 661/1000
 - 0s - loss: 2.9282e-04 - acc: 1.0000 - val_loss: 0.4030 - val_acc: 0.9750
Epoch 662/1000
 - 0s - loss: 0.0116 - acc: 0.9972 - val_loss: 0.3517 - val_acc: 0.9750
Epoch 663/1000
 - 0s - loss: 0.0012 - acc: 1.0000 - val_loss: 0.4030 - val_acc: 0.9750
Epoch 664/1000
 - 0s - loss: 0.0180 - acc: 0.9981 - val_loss: 0.3781 - val_acc: 0.9750
Epoch 665/1000
 - 0s - loss: 0.0189 - acc: 0.9981 - val_loss: 0.4030 - val_acc: 0.9750
Epoch 666/1000
 - 0s - loss: 0.0070 - acc: 0.9981 - val_loss: 0.3220 - val_acc: 0.9750
Epoch 667/1000
 - 0s - loss: 0.0184 - acc: 0.9972 - val_loss: 0.2051 - val_acc: 0.9833
Epoch 668/1000
 - 0s - loss: 0.0046 - acc:

Epoch 750/1000
 - 0s - loss: 0.0151 - acc: 0.9963 - val_loss: 0.2997 - val_acc: 0.9750
Epoch 751/1000
 - 0s - loss: 0.0095 - acc: 0.9972 - val_loss: 0.3222 - val_acc: 0.9667
Epoch 752/1000
 - 0s - loss: 0.0011 - acc: 1.0000 - val_loss: 0.3394 - val_acc: 0.9667
Epoch 753/1000
 - 0s - loss: 0.0036 - acc: 0.9981 - val_loss: 0.3014 - val_acc: 0.9750
Epoch 754/1000
 - 0s - loss: 7.4354e-04 - acc: 1.0000 - val_loss: 0.2619 - val_acc: 0.9750
Epoch 755/1000
 - 0s - loss: 2.1411e-04 - acc: 1.0000 - val_loss: 0.3335 - val_acc: 0.9750
Epoch 756/1000
 - 0s - loss: 0.0150 - acc: 0.9991 - val_loss: 0.3675 - val_acc: 0.9750
Epoch 757/1000
 - 0s - loss: 0.0034 - acc: 0.9991 - val_loss: 0.3203 - val_acc: 0.9750
Epoch 758/1000
 - 0s - loss: 0.0157 - acc: 0.9991 - val_loss: 0.2639 - val_acc: 0.9750
Epoch 759/1000
 - 0s - loss: 0.0321 - acc: 0.9963 - val_loss: 0.4030 - val_acc: 0.9750
Epoch 760/1000
 - 0s - loss: 5.3999e-04 - acc: 1.0000 - val_loss: 0.4030 - val_acc: 0.9750
Epoch 761/1000
 - 0s - loss: 6.

Epoch 844/1000
 - 0s - loss: 0.0084 - acc: 0.9963 - val_loss: 0.3434 - val_acc: 0.9750
Epoch 845/1000
 - 0s - loss: 0.0068 - acc: 0.9981 - val_loss: 0.3444 - val_acc: 0.9750
Epoch 846/1000
 - 0s - loss: 0.0033 - acc: 0.9981 - val_loss: 0.4030 - val_acc: 0.9750
Epoch 847/1000
 - 0s - loss: 0.0180 - acc: 0.9981 - val_loss: 0.4030 - val_acc: 0.9750
Epoch 848/1000
 - 0s - loss: 0.0026 - acc: 0.9981 - val_loss: 0.4030 - val_acc: 0.9750
Epoch 849/1000
 - 0s - loss: 0.0058 - acc: 0.9981 - val_loss: 0.4030 - val_acc: 0.9750
Epoch 850/1000
 - 0s - loss: 0.0075 - acc: 0.9981 - val_loss: 0.3359 - val_acc: 0.9750
Epoch 851/1000
 - 0s - loss: 0.0053 - acc: 0.9981 - val_loss: 0.4030 - val_acc: 0.9750
Epoch 852/1000
 - 0s - loss: 0.0154 - acc: 0.9991 - val_loss: 0.4030 - val_acc: 0.9750
Epoch 853/1000
 - 0s - loss: 0.0229 - acc: 0.9981 - val_loss: 0.3702 - val_acc: 0.9750
Epoch 854/1000
 - 0s - loss: 0.0167 - acc: 0.9981 - val_loss: 0.3334 - val_acc: 0.9750
Epoch 855/1000
 - 0s - loss: 0.0043 - acc: 

 - 0s - loss: 0.0325 - acc: 0.9926 - val_loss: 0.4084 - val_acc: 0.9750
Epoch 939/1000
 - 0s - loss: 0.0112 - acc: 0.9991 - val_loss: 0.4838 - val_acc: 0.9667
Epoch 940/1000
 - 0s - loss: 0.0116 - acc: 0.9963 - val_loss: 0.4102 - val_acc: 0.9667
Epoch 941/1000
 - 0s - loss: 0.0191 - acc: 0.9981 - val_loss: 0.4073 - val_acc: 0.9750
Epoch 942/1000
 - 0s - loss: 0.0167 - acc: 0.9991 - val_loss: 0.4075 - val_acc: 0.9750
Epoch 943/1000
 - 0s - loss: 0.0154 - acc: 0.9991 - val_loss: 0.4076 - val_acc: 0.9750
Epoch 944/1000
 - 0s - loss: 0.0035 - acc: 0.9981 - val_loss: 0.4079 - val_acc: 0.9750
Epoch 945/1000
 - 0s - loss: 0.0061 - acc: 0.9991 - val_loss: 0.4068 - val_acc: 0.9750
Epoch 946/1000
 - 0s - loss: 0.0106 - acc: 0.9981 - val_loss: 0.4058 - val_acc: 0.9750
Epoch 947/1000
 - 0s - loss: 3.2759e-04 - acc: 1.0000 - val_loss: 0.4055 - val_acc: 0.9750
Epoch 948/1000
 - 0s - loss: 0.0031 - acc: 0.9981 - val_loss: 0.4055 - val_acc: 0.9750
Epoch 949/1000
 - 0s - loss: 0.0060 - acc: 0.9972 - va

In [9]:
scores = model.evaluate(x_test_pca, y_testOneHot)
scores[1]

300/300 [==============================] - 0s 1ms/step


0.6833333325386047

In [10]:

prediction = model.predict_classes(x_test_pca)
pd.crosstab(y_test.iloc[:,1],prediction,
            rownames=['True'],colnames=['Predict'])

Predict,0,1
True,,
0,181,62
1,33,24


In [11]:
#### confusion table

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


names = ["Healthy eyes", "Dr Disease"]
print(classification_report(y_test.iloc[:,1], prediction, target_names = names))
#print(confusion_matrix(y_test, prediction, labels=[0,1]))


              precision    recall  f1-score   support

Healthy eyes       0.85      0.74      0.79       243
  Dr Disease       0.28      0.42      0.34        57

    accuracy                           0.68       300
   macro avg       0.56      0.58      0.56       300
weighted avg       0.74      0.68      0.71       300

